In [ ]:
#inherit parent class: tf.keras.Model
class Encoder(tf.keras.Model):
    # input params for the class
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        
        # calling parent initiator from it has been inheried
        # tf.keras.Model is being iniatiated
        super(Encoder, self).__init__()

        # initiating params
        self.batch_sz = batch_sz
        self.enc_units = enc_units

        # embeding is the 1st layer of encoder network
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        # iniating GRU. We can use RNN or LSTM also
        # return_sequences: return output of each state i.e., each h0
        # return_state: the final context
        self.gru = tf.keras.layers.GRU(self.enc_units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
    
    # building the above model
    # x: input
    # hidden: initial hidden state (h0)
    def call(self, x, hidden):
        # build embeding layer
        x = self.embedding(x)

        # pass embeding layer to GRU
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))


# calling encoder
# vocab_inp_size: len of uniuque words in output language / input size for the GRU network
# units: no.of hidden units, it's a hyperparameter
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
#inherit parent class: tf.keras.Model
class Decoder(tf.keras.Model):

    # input params for the class
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):

        # calling parent initiator from it has been inheried
        # tf.keras.Model is being iniatiated
        super(Decoder, self).__init__()

        # initiating params
        self.batch_sz = batch_sz
        self.dec_units = dec_units

        # embeding is the 1st layer of encoder network
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        # iniating GRU. We can use RNN or LSTM also
        # return_sequences: return output of each state i.e., each h0
        # return_state: the final context
        self.gru = tf.keras.layers.GRU(self.dec_units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
        
        # initialization of attention mechanism
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        # weights for the inputs, as discussed in attention theory
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)

        # output of attention
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape: (batch_size, max_length, hidden_size)
        
        # hidden shape: (batch_size, hidden size)
        # hidden_with_time_axis shape: (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score for attention mechanism 
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape: (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape: (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum: (batch_size, hidden_size)
        # weighted calculation: multiply weights and hidden state outputs
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding: (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation: (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape: (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape: (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

# calling decoder
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
import time

EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            # input to the decoder network
            dec_hidden = enc_hidden
            
            # triggering 1st input of decoder with <start> tag for each batch
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))